In [1]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
from ast import literal_eval
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from keras.layers import *
from sklearn.model_selection import cross_val_score 

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Remove the rows which contains values ['No resposne']

In [4]:
import pandas as pd
df=pd.read_csv('isear.csv',header=None)
print(len(df))

7652


In [5]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
df.drop(df[df[1] == '[ No response.]'].index, inplace = True)
print(len(df))

7575


In [7]:
index_names = df[ df[0] == 'shame' ].index 
df.drop(index_names, inplace = True) 

In [8]:
#taking into consideration only four emotions- joy,fear,sadness,anger
index_names = df[ df[0] == 'disgust' ].index 
df.drop(index_names, inplace = True) 


In [9]:
index_names = df[ df[0] == 'guilt' ].index 
df.drop(index_names, inplace = True) 

In [10]:
print(len(df))

4344


In [11]:
df.reset_index(inplace = True) 
feel_arr=df[1]
print(feel_arr)

0       [ On days when I feel close to my partner and ...
1       Every time I imagine that someone I love or I ...
2       When I had been obviously unjustly treated and...
3       When I think about the short time that we live...
4       After my girlfriend had taken her exam we went...
                              ...                        
4339    This issue worried me rather than saddening me...
4340           I received a letter from a distant friend.
4341    My parents were out and I was the eldest at ho...
4342    Two years back someone invited me to be the tu...
4343    I had taken the responsibility to do something...
Name: 1, Length: 4344, dtype: object


In [12]:
feel_arr=df[1]
feel_arr[0]

'[ On days when I feel close to my partner and other friends.  \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.]'

In [13]:
feel_arr=[word_tokenize(sent) for sent in feel_arr]
print(feel_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']']


In [14]:
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('<pad>')
    return arr[:100]
for i in range(len(feel_arr)):
    feel_arr[i]=padd(feel_arr[i])

In [16]:
vocab_f=open('../glove.6B.50d.txt',encoding="utf-8")
word_to_emb={}
for line in vocab_f:
    word_to_emb[line.split()[0]]=[float(i) for i in line.split()[1:]] 
word_to_emb['<pad>']=[0]*50
print(len(word_to_emb['happy']))

50


In [17]:
embedded_feel_arr=[] 
for each_sentence in feel_arr:
    embedded_feel_arr.append([])
    for word in each_sentence:
        if word.lower() in word_to_emb:
            embedded_feel_arr[-1].append(word_to_emb[word.lower()])
        else:
            embedded_feel_arr[-1].append([0]*50)
print(embedded_feel_arr[0])

[[-0.61201, 0.98226, 0.11539, 0.014623, 0.23873, -0.067035, 0.30632, -0.64742, -0.38517, -0.03691, 0.094788, 0.57631, -0.091557, -0.54825, 0.25255, -0.14759, 0.13023, 0.21658, -0.30623, 0.30028, -0.23471, -0.17927, 0.9518, 0.54258, 0.31172, -0.51038, -0.65223, -0.48858, 0.13486, -0.40132, 2.493, -0.38777, -0.26456, -0.49414, -0.3871, -0.20983, 0.82941, -0.46253, 0.39549, 0.014881, 0.79485, -0.79958, -0.16243, 0.013862, -0.53536, 0.52536, 0.019818, -0.16353, 0.30649, 0.81745], [0.30045, 0.25006, -0.16692, 0.1923, 0.026921, -0.079486, -0.91383, -0.1974, -0.053413, -0.40846, -0.26844, -0.28212, -0.5, 0.1221, 0.3903, 0.17797, -0.4429, -0.40478, -0.9505, -0.16897, 0.77793, 0.33525, 0.3346, -0.1754, -0.12017, -1.7861, 0.29241, 0.55933, 0.029982, -0.32417, 3.9297, 0.1088, -0.57335, -0.17842, 0.0041748, -0.16309, 0.45077, -0.16123, -0.17311, -0.087889, -0.089032, 0.062001, -0.19946, -0.38863, -0.18232, 0.060751, 0.098603, -0.07131, 0.23052, -0.51939], [0.62236, 0.1973, 0.0023326, -0.44924, 0.0

In [18]:
X=np.array(embedded_feel_arr)
print(np.shape(X))

(4344, 100, 50)


In [19]:
Y= df[0].unique()
print(Y)

['joy' 'fear' 'anger' 'sadness']


In [20]:
y_map = {} 
for i in range(len(Y)):
  y_map[Y[i]] = i
print(y_map)

{'joy': 0, 'fear': 1, 'anger': 2, 'sadness': 3}


In [21]:
df[0] = df[0].map(y_map)

In [22]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
res = enc.fit_transform(np.array(df[0]).reshape(-1,1)).toarray()
print(res.shape)

(4344, 4)


In [23]:
Y= res
print(Y)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [24]:
print(type(Y))

<class 'numpy.ndarray'>


In [25]:
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [26]:
def model1(X,Y,input_size1,input_size2,output_size):
  m=Sequential()
  #m.add(Embedding(7575, 7, input_length=1,trainable=True, embeddings_initializer='uniform'))
  m.add(Bidirectional(LSTM(100,input_shape=(input_size1,input_size2))))
  m.add(Dropout(0.5))
  #m.add(Dense(50,activation='relu'))
  #m.add(Dropout(0.2))
  m.add(Dense(output_size,activation='softmax'))
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=32, batch_size=128)
  return m

In [27]:
bilstmModel=model1(X_train,Y_train,100,50,4)

Epoch 1/32
31/31 [==============================] - 10s 326ms/step - loss: 1.3595 - accuracy: 0.3239
Epoch 2/32
31/31 [==============================] - 10s 319ms/step - loss: 1.1841 - accuracy: 0.4978
Epoch 3/32
31/31 [==============================] - 10s 312ms/step - loss: 1.0657 - accuracy: 0.5631
Epoch 4/32
31/31 [==============================] - 9s 305ms/step - loss: 1.0238 - accuracy: 0.5815
Epoch 5/32
31/31 [==============================] - 10s 316ms/step - loss: 0.9629 - accuracy: 0.6150
Epoch 6/32
31/31 [==============================] - 11s 350ms/step - loss: 0.9450 - accuracy: 0.6283
Epoch 7/32
31/31 [==============================] - 11s 355ms/step - loss: 0.8996 - accuracy: 0.6480
Epoch 8/32
31/31 [==============================] - 11s 365ms/step - loss: 0.8889 - accuracy: 0.6513
Epoch 9/32
31/31 [==============================] - 12s 373ms/step - loss: 0.8759 - accuracy: 0.6557
Epoch 10/32
31/31 [==============================] - 11s 370ms/step - loss: 0.8357 - accurac

In [28]:
bilstmModel.evaluate(X_test,Y_test)

14/14 [==============================] - 0s 32ms/step - loss: 0.8729 - accuracy: 0.6966


[0.872896671295166, 0.6965517401695251]

In [29]:
y_inverse_map={}
for i in y_map:
  y_inverse_map[y_map[i]]=i
print(y_inverse_map)

{0: 'joy', 1: 'fear', 2: 'anger', 3: 'sadness'}


In [30]:
def prepare(s):
  s=word_tokenize(s)
  #print("s= ",s)
  k=padd(s)
  embedded_arr=[]
  embedded_arr.append([])
  for word in k:
        if word.lower() in word_to_emb:
            embedded_arr[-1].append(word_to_emb[word.lower()])
        else:
            embedded_arr[-1].append([0]*50)
  x_input=np.array(embedded_arr)
  #print(np.shape(x_input))
  arr = bilstmModel.predict(x_input)
  return arr.argmax()

In [ ]:
s=input()
emotion=prepare(s)
print(y_inverse_map[emotion])

lets have some fun tonight
joy


In [31]:
bilstmModel.save('lstm_model.h5')